In [1]:
# Import libraries
from PyPDF2 import PdfReader
from tqdm.notebook import tqdm
from textblob import TextBlob
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import pdfplumber
import re
import os

In [2]:
# load data
netflix_df = pd.read_csv('Src/NFLX.csv')
# find the daily return
netflix_df["Daily Return"] = netflix_df["Close"].pct_change()
# drop columns
netflix_df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)

In [3]:
netflix_df.head()

,Date,Close,Daily Return
0,2013-06-17,32.747143,NaN
1,2013-06-18,32.689999,-0.001745
2,2013-06-19,33.187141,0.015208
3,2013-06-20,31.931429,-0.037837
4,2013-06-21,30.985714,-0.029617


In [ ]:
folder = 'ShareholderLetters'

report_dates = {}
main_pattern = r"([a-zA-Z]+) ([0-9]+), ([0-9]+)"
recovery_pattern = r"([a-zA-Z]+)([0-9]+),([0-9]+)"

# iterate through all files in the folder and extract report dates
for file in tqdm(os.listdir(folder)):
    pdf = pdfplumber.open(folder + "/" + file)
    report_date = pdf.pages[0].extract_text().split('\n')[0]
    if re.search(main_pattern, report_date):
        report_dates[file] = report_date
    else:
        report_date = re.sub(recovery_pattern, r"\1 \2, \3", report_date)
        report_dates[file] = report_date

In [27]:
# create a dataframe from the dictionary
report_dates_df = pd.DataFrame.from_dict(report_dates, orient='index', columns=['Report_Date'])
# format the date columns in dataframes
report_dates_df['Report_Date'] = pd.to_datetime(report_dates_df['Report_Date'], format='%B %d, %Y')
netflix_df['Date'] = pd.to_datetime(netflix_df['Date'])
# merge the two dataframes
total_df = pd.merge(netflix_df, report_dates_df, left_on='Date', right_on='Report_Date', how='left')

total_df["Day 1"] = total_df["Report_Date"] + timedelta(days=1)
total_df["Day 1 return"] = total_df["Daily Return"].shift(-1)
total_df["Day 2"] = total_df["Report_Date"] + timedelta(days=2)
total_df["Day 2 return"] = total_df["Daily Return"].shift(-2)
total_df["Day 3"] = total_df["Report_Date"] + timedelta(days=3)
total_df["Day 3 return"] = total_df["Daily Return"].shift(-3)
total_df["Average 3-Day return"] = (total_df["Day 1 return"] + total_df["Day 2 return"] + total_df["Day 3 return"]) / 3

In [28]:
sentiment_scores_df = pd.read_excel('Sentiment Scores.xlsx')
sentiment_scores_df['Report_Date'] = pd.to_datetime(sentiment_scores_df['Report_Date'])
# merge the two dataframes
total_df = pd.merge(total_df, sentiment_scores_df, left_on='Report_Date', right_on='Report_Date', how='left')

In [29]:
anr_df = pd.read_excel('ANR.xlsx', sheet_name='Sheet2')

In [30]:
anr_df.head()

,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year
0,2023-04-01,2024-03-31,366.69,2023-04-01,345.48
1,2023-03-25,2024-03-24,366.69,2023-04-01,345.48
2,2023-03-18,2024-03-17,365.15,2023-04-01,345.48
3,2023-03-11,2024-03-10,363.36,2023-04-01,345.48
4,2023-03-04,2024-03-03,364.64,2023-04-01,345.48


In [31]:
anr_df['ANR_Date'] = pd.to_datetime(anr_df['ANR_Date'])
# shift the ANR date by 1 day before
anr_df['ANR_Date'] = anr_df['ANR_Date'] - timedelta(days=1)
# merge the two dataframes
total_df = pd.merge(total_df, anr_df, left_on='Date', right_on='ANR_Date', how='left')

In [77]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs((x - pivot)))

In [79]:
total_df["Report_Date"].fillna(0, inplace=True)
total_df["Closest date"].fillna(0, inplace=True)

In [80]:
total_df["Report_Date"] = pd.to_datetime(total_df["Report_Date"])
total_df["Closest date"] = pd.to_datetime(total_df["Closest date"])

In [81]:
total_df['Closest_Report_Date'] = total_df.apply(lambda x: nearest(total_df['Closest date'], x['Report_Date']), axis=1)

In [102]:
total_df[~ total_df["Report"].isnull()].T

,2226,2288,2352,2415
Date,2022-04-19 00:00:00,2022-07-19 00:00:00,2022-10-18 00:00:00,2023-01-19 00:00:00
Close,348.609985,201.630005,240.860001,315.779999
Daily Return,0.031818,0.056097,-0.017299,-0.032329
Report_Date,2022-04-19 00:00:00,2022-07-19 00:00:00,2022-10-18 00:00:00,2023-01-19 00:00:00
Day 1,2022-04-20 00:00:00,2022-07-20 00:00:00,2022-10-19 00:00:00,2023-01-20 00:00:00
Day 1 return,-0.351166,0.073451,0.130864,0.084616
Day 2,2022-04-21 00:00:00,2022-07-21 00:00:00,2022-10-20 00:00:00,2023-01-21 00:00:00
Day 2 return,-0.035236,0.034374,-0.015493,0.043562
Day 3,2022-04-22 00:00:00,2022-07-22 00:00:00,2022-10-21 00:00:00,2023-01-22 00:00:00
Day 3 return,-0.012373,-0.015365,0.07984,0.017934


In [96]:
total_df.drop(columns=["Closest date_y", "Actual price in 1 year_y", "Closest date_x"], inplace=True)

In [98]:
last_df = anr_df[["Closest date", "Actual price in 1 year", "Target price estimate in 1 year"]]

In [93]:
anr_df

,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year
0,2023-03-31,2024-03-31,366.69,2023-04-01,345.48
1,2023-03-24,2024-03-24,366.69,2023-04-01,345.48
2,2023-03-17,2024-03-17,365.15,2023-04-01,345.48
3,2023-03-10,2024-03-10,363.36,2023-04-01,345.48
4,2023-03-03,2024-03-03,364.64,2023-04-01,345.48
...,...,...,...,...,...
767,2008-07-18,2009-07-19,4.77,2009-07-18,6.31
768,2008-07-11,2009-07-12,4.77,2009-07-11,5.72
769,2008-07-04,2009-07-05,4.77,2009-07-04,5.70
770,2008-06-27,2009-06-28,4.82,2009-06-27,5.77


In [100]:
last_df["Closest date"] = pd.to_datetime(last_df["Closest date"])
total_df["Closest_Report_Date"] = pd.to_datetime(total_df["Closest_Report_Date"])

C:\Users\Я\AppData\Local\Temp\ipykernel_17484\2991060944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df["Closest date"] = pd.to_datetime(last_df["Closest date"])


In [101]:
total_df = pd.merge(total_df, last_df, left_on='Closest_Report_Date', right_on='Closest date', how='left')

In [ ]:
total_df["Actual price in 1 year"] = anr_df[anr_df["Closest date"] == total_df["Closest_Report_Date"]]["Actual price in 1 year"].values

In [83]:
total

,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year
0,2023-03-31,2024-03-31,366.69,2023-04-01,345.48
1,2023-03-24,2024-03-24,366.69,2023-04-01,345.48
2,2023-03-17,2024-03-17,365.15,2023-04-01,345.48
3,2023-03-10,2024-03-10,363.36,2023-04-01,345.48
4,2023-03-03,2024-03-03,364.64,2023-04-01,345.48
...,...,...,...,...,...
767,2008-07-18,2009-07-19,4.77,2009-07-18,6.31
768,2008-07-11,2009-07-12,4.77,2009-07-11,5.72
769,2008-07-04,2009-07-05,4.77,2009-07-04,5.70
770,2008-06-27,2009-06-28,4.82,2009-06-27,5.77


In [60]:
check_df = total_df[['Report_Date', 'Closest date']]

In [61]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Report_Date   20 non-null     datetime64[ns]
 1   Closest date  494 non-null    datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 39.5 KB


In [65]:
check_df.fillna(0, inplace=True)

C:\Users\Я\AppData\Local\Temp\ipykernel_17484\3598655241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df.fillna(0, inplace=True)


In [66]:
check_df['Report_Date'] = pd.to_datetime(check_df['Report_Date'])
check_df['Closest date'] = pd.to_datetime(check_df['Closest date'])

C:\Users\Я\AppData\Local\Temp\ipykernel_17484\1532036003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['Report_Date'] = pd.to_datetime(check_df['Report_Date'])
C:\Users\Я\AppData\Local\Temp\ipykernel_17484\1532036003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['Closest date'] = pd.to_datetime(check_df['Closest date'])


In [73]:
check_df['Closest'] = check_df.apply(lambda x: nearest(check_df['Closest date'], x['Report_Date']), axis=1)

C:\Users\Я\AppData\Local\Temp\ipykernel_17484\3384942226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df['Closest'] = check_df.apply(lambda x: nearest(check_df['Closest date'], x['Report_Date']), axis=1)


In [74]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Report_Date   2517 non-null   datetime64[ns]
 1   Closest date  2517 non-null   datetime64[ns]
 2   Closest       2517 non-null   datetime64[ns]
dtypes: datetime64[ns](3)
memory usage: 59.1 KB


In [75]:
check_df[check_df['Report_Date'] != "1970-01-01"]

,Report_Date,Closest date,Closest
965,2017-04-17,1970-01-01,2017-04-15
1028,2017-07-17,1970-01-01,2017-07-15
1092,2017-10-16,1970-01-01,2017-10-14
1158,2018-01-22,1970-01-01,2018-01-20
1531,2019-07-17,1970-01-01,2019-07-20
1595,2019-10-16,1970-01-01,2019-10-19
1660,2020-01-21,1970-01-01,2020-01-18
1723,2020-04-21,1970-01-01,2020-04-25
1783,2020-07-16,1970-01-01,2020-07-18
1850,2020-10-20,1970-01-01,2020-10-17


In [12]:
anr_df.head()

,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year
0,2023-03-31,2024-03-31,366.69,2023-04-01,345.48
1,2023-03-24,2024-03-24,366.69,2023-04-01,345.48
2,2023-03-17,2024-03-17,365.15,2023-04-01,345.48
3,2023-03-10,2024-03-10,363.36,2023-04-01,345.48
4,2023-03-03,2024-03-03,364.64,2023-04-01,345.48


In [58]:
total_df[~ total_df["Report"].isnull()].T

,2226,2288,2352,2415
Date,2022-04-19 00:00:00,2022-07-19 00:00:00,2022-10-18 00:00:00,2023-01-19 00:00:00
Close,348.609985,201.630005,240.860001,315.779999
Daily Return,0.031818,0.056097,-0.017299,-0.032329
Report_Date,2022-04-19 00:00:00,2022-07-19 00:00:00,2022-10-18 00:00:00,2023-01-19 00:00:00
Day 1,2022-04-20 00:00:00,2022-07-20 00:00:00,2022-10-19 00:00:00,2023-01-20 00:00:00
Day 1 return,-0.351166,0.073451,0.130864,0.084616
Day 2,2022-04-21 00:00:00,2022-07-21 00:00:00,2022-10-20 00:00:00,2023-01-21 00:00:00
Day 2 return,-0.035236,0.034374,-0.015493,0.043562
Day 3,2022-04-22 00:00:00,2022-07-22 00:00:00,2022-10-21 00:00:00,2023-01-22 00:00:00
Day 3 return,-0.012373,-0.015365,0.07984,0.017934


In [22]:
total_df[total_df["Date"] > "2022-04-15"]

,Date,Close,Daily Return,Report_Date,Day 1,Day 1 return,Day 2,Day 2 return,Day 3,Day 3 return,...,Subjectivity Score,Answer Score,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year,Report_Date_ordinal,Closest date_ordinal,closest_date_new


In [17]:
total_df['Report_Date'] = pd.to_datetime(total_df['Report_Date'])
total_df['Closest date'] = pd.to_datetime(total_df['Closest date'])

In [21]:
total_df

,Date,Close,Daily Return,Report_Date,Day 1,Day 1 return,Day 2,Day 2 return,Day 3,Day 3 return,...,Subjectivity Score,Answer Score,ANR_Date,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year,Report_Date_ordinal,Closest date_ordinal,closest_date_new


In [22]:
total_df[~ total_df["Date in 1 year"].isnull()]

,Date,Close,Daily Return,Report_Date,Day 1,Day 1 return,Day 2,Day 2 return,Day 3,Day 3 return,Average 3-Day return,Report,Polarity Score,Subjectivity Score,Answer Score,Date in 1 year,Target price estimate in 1 year,Closest date,Actual price in 1 year
